<a href="https://colab.research.google.com/github/ElahehJafarigol/Federated-Learning-GANs/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Deep Imbalanced Learning for Weather Data: A Federated Learning Approach**

###1. Traditional GANs without Label conditioning

In [ ]:
!pip install --quiet keras==2.9.0
!pip install --quiet tensorflow==2.9.2

In [ ]:
!pip install --quiet imbalanced-learn

In [ ]:
import os
from  IPython import display
import pathlib
import shutil
import tempfile
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
from pandas.core.common import random_state
import pandas as pd
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

random_state = 42
random_seed = 42
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras import backend as K
from keras import datasets, layers, models, metrics
from keras.layers import GaussianNoise
from keras import regularizers

from keras.layers.serialization import activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution
from keras.layers import Concatenate

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import roc_auc_score, confusion_matrix


####**Multiple Datasets**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving WeatherAUS.csv to WeatherAUS (2).csv
User uploaded file "WeatherAUS.csv" with length 14159645 bytes


In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded['WeatherAUS.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

print(Data.shape)
Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

counter = Counter(y)
print(counter)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))

X = pd.DataFrame(X.reset_index(drop=True))

original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X_complete = X
y_complete = y

In [ ]:
X = X_complete
y = y_complete

#####Data Augmentation

In [ ]:
from pickle import FALSE
epochs = 10
batch_size = 64
latent_dim = 13

# Define the generator model
def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(1024, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(X.shape[1], activation='sigmoid'))

    # model.summary()
    return model

# Define the discriminator model
def define_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.SGD(0.0003, 0.5),
                  metrics=['accuracy'])
    # model.summary()
    return model

# Train the CGANs model
def train(X, y, epochs, batch_size, latent_dim):

    # Remove rows with missing data
    X = X.dropna()

    # Get the set of valid indices
    valid_indices = set(X.index)

    # Instantiate the generator and Discriminator
    generator = define_generator(latent_dim)
    discriminator = define_discriminator(X.shape[1])

    # Freeze discriminator weights
    """One crucial detail in the code below is that we make the Discriminator model non-trainable.
     We do this because we want to train the Discriminator separately using a combination of
     real and fake (generated) data."""
    discriminator.trainable = False

    # Define the cGANs model
    cgan_input = Input(shape=(latent_dim,))
    discriminator_output = discriminator(generator(cgan_input))
    cgan = Model(cgan_input, discriminator_output)
    cgan.compile(loss='binary_crossentropy',
                 optimizer=keras.optimizers.SGD(0.0003, 0.5))

    # Train the CGANs model
    for epoch in range(epochs):

        # Train the discriminator
        """The first code snippet is better for training the discriminator in a Wasserstein GAN.
        The reason is that in a Wasserstein GAN, the discriminator is trained to optimize
        a distance measure between the real and generated data distributions,
        while in a traditional GAN, the discriminator is trained to classify real and fake samples.
        To achieve this, the discriminator needs to be trained multiple times on both
        real and generated data. In the first code snippet, the discriminator is
        trained for n_critic iterations on both real and generated data.
        This helps the discriminator to better estimate the Wasserstein distance
        between the two distributions."""

        try:
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_data = generator.predict(noise)

            # Sample indices from the valid range of indices in the dataset
            real_indices = np.random.choice(list(valid_indices), size=batch_size, replace=False)
            real_data = X.loc[real_indices]
            combined_data = np.concatenate([generated_data, real_data])
            labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])
            d_loss = discriminator.train_on_batch(combined_data, labels)

            # Clip discriminator weights
            for layer in discriminator.layers:
                weights = layer.get_weights()
                weights = [np.clip(w, -0.01, 0.01) for w in weights]
                layer.set_weights(weights)

        except KeyError as e:
            print(f"Skipping row {id} due to KeyError: {e}")
            continue

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.ones((batch_size, 1))
        g_loss = cgan.train_on_batch(noise, misleading_targets)

        # Print the progress
        print(f"Epoch {epoch} Discriminator Loss: {d_loss} Generator Loss: {g_loss}")

In [ ]:
train(X, y, epochs = 20, batch_size = 128, latent_dim = 13)

4/4 [==============================] - 0s 8ms/step
Epoch 0 Discriminator Loss: [0.5881823301315308, 0.58984375] Generator Loss: 0.6931891441345215
4/4 [==============================] - 0s 6ms/step
Epoch 1 Discriminator Loss: [0.6930909752845764, 0.515625] Generator Loss: 0.6932088136672974
4/4 [==============================] - 0s 6ms/step
Epoch 2 Discriminator Loss: [0.6931594014167786, 0.55078125] Generator Loss: 0.6931636929512024
4/4 [==============================] - 0s 6ms/step
Epoch 3 Discriminator Loss: [0.6930504441261292, 0.5859375] Generator Loss: 0.6931666731834412
4/4 [==============================] - 0s 6ms/step
Epoch 4 Discriminator Loss: [0.6930793523788452, 0.5390625] Generator Loss: 0.6931726932525635
4/4 [==============================] - 0s 5ms/step
Epoch 5 Discriminator Loss: [0.6931161880493164, 0.5] Generator Loss: 0.6932035684585571
4/4 [==============================] - 0s 5ms/step
Epoch 6 Discriminator Loss: [0.6932359933853149, 0.5] Generator Loss: 0.693178

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

In [ ]:
# Save the generator
generator_00 = define_generator(latent_dim)

# Generate synthetic data using the GAN generator
n_samples = len(X_train)
synthetic_data = generator_00.predict(np.random.normal(0, 1, (n_samples, latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced_0 = np.concatenate([X_train, synthetic_data])
y_balanced_0 = np.concatenate([y_train, np.zeros(n_samples)])

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_balanced_0 == 0) / len(y_balanced_0)
print(class_ratio)

print(X_balanced_0.shape)
print(X_train.shape)

3555/3555 [==============================] - 14s 4ms/step


0.8877577931325492

In [ ]:
df_0 = pd.DataFrame(X_balanced_0)
df_0['RainTomorrow'] = y_balanced_0

# count target class
#sn.countplot(x='RainTomorrow', data=df_0,  palette = "RdBu")

#####Training Federated Learning

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
def create_clients(X_train, y_train, num_clients, initial='client'):
    # create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    # randomize the data before splitting the data between clients
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]

    # shard data and place at each client
    size = len(X_train) // num_clients
    remainder = len(X_train) % num_clients
    shards = [(X_shuffled[i:i+size], y_shuffled[i:i+size]) for i in range(0, size*num_clients, size)]
    if remainder:
        shards[-1] = (np.concatenate([shards[-1][0], X_shuffled[-remainder:]]),
                      np.concatenate([shards[-1][1], y_shuffled[-remainder:]]))

    # number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    # create dictionary of clients and their shards
    clients = {}
    for i in range(len(client_names)):
        # create dictionary entry with client name and data shape
        clients[client_names[i]] = (shards[i][0].shape, shards[i][1].shape)

    return clients

combined_data_shard = (combined_X_test, combined_y_test)
test_data_batched = batch_data(combined_data_shard, batchsize=32)


# create clients from training data and associated labels
X_train = X_balanced_0
y_train = y_balanced_0
clients = create_clients(X_train, y_train, num_clients=4, initial='client')
for client in clients:
    print(f"{client}: X={clients[client][0]}, y={clients[client][1]}")

# You can access the shards for each client by indexing the dictionary returned
# by create_clients(), like so:
client_1_shard = clients['client_1']
client_1_shard

# create a batched dataset for each client's data shard
clients_batched = {}
for client in clients:
    client_data_shard = (X_train, y_train)
    batched_data = batch_data(client_data_shard)
    clients_batched[client] = batched_data

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

# Federated Averaging: aggregation method
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the batch_size
    batch_size = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*batch_size
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*batch_size
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    # Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

def test_model(X_test, Y_test, model, communication_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=Batch_size)
    logits = tf.squeeze(logits, axis=1) # remove last dimension from logits
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.round(logits), Y_test)
    auc = roc_auc_score(Y_test, logits)
    tn, fp, fn, tp = confusion_matrix(Y_test, tf.round(logits)).ravel()
    g_mean = np.sqrt(tp/(tp+fn)*tn/(tn+fp))
    print('communication_round: {} | global_accuracy: {:.3%} | global_loss: {} | global_AUC: {:.3%} | global_G-mean: {:.3%}'.format(communication_round, acc, loss, auc, g_mean))
    return acc, loss, auc, g_mean

In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      ]

class MyModel:
    @staticmethod
    def build(shape, classes):
        model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(13,)),
        layers.GaussianNoise(stddev = 0.5),
        layers.Dense(128, activation='relu'),
        layers.GaussianNoise(stddev = 0.3),
        layers.Dense(64, activation='relu'),
        layers.GaussianNoise(stddev = 0.1),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
        ])
        return model

In [ ]:
smlp_global = MyModel()
global_model = smlp_global.build(shape=(13,), classes=2)
global_model.summary()

global_acc_list = []
global_loss_list = []
global_auc_list = []
global_gmean_list = []

# commence global training loop
for communication_round in range(communication_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # loop through each client and create new local model
    for client in client_names:
        smlp_local = MyModel()
        local_model = smlp_local.build(shape=(13,), classes=2)
        #local_model = smlp_local.build()
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)

        # set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        model_history = local_model.fit(clients_batched[client], epochs=epochs, verbose=1)

        # evaluate local model on test data
        # test_loss, test_accuracy = local_model.evaluate(X_test, y_test)
        # print(f'Local model {client} - test_loss: {test_loss} - test_accuracy: {test_accuracy}')

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # clear session to free memory after each communication round
        K.clear_session()

    # to get the average over all the local model, we simply take the sum of the scaled weights
    avg_grad = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(avg_grad)

    # test global model and print out metrics after each communications round
    for X_test, y_test in test_batched:
        global_acc, global_loss, global_auc, global_gmean = test_model(X_test, y_test, global_model, communication_round)
        global_acc_list.append(global_acc)
        global_loss_list.append(global_loss)
        global_auc_list.append(global_auc)
        global_gmean_list.append(global_gmean)

Epoch 1/30
3555/3555 [==============================] - 20s 5ms/step - loss: 0.1982 - accuracy: 0.9157 - precision: 0.6981 - recall: 0.4380 - auc: 0.9249 - prc: 0.6364 - tp: 11184.0000 - fp: 4837.0000 - tn: 197135.0000 - fn: 14352.0000
Epoch 2/30
3555/3555 [==============================] - 21s 6ms/step - loss: 0.1897 - accuracy: 0.9188 - precision: 0.7199 - recall: 0.4534 - auc: 0.9320 - prc: 0.6629 - tp: 11577.0000 - fp: 4505.0000 - tn: 197467.0000 - fn: 13959.0000
Epoch 3/30
3555/3555 [==============================] - 18s 5ms/step - loss: 0.1874 - accuracy: 0.9196 - precision: 0.7285 - recall: 0.4522 - auc: 0.9338 - prc: 0.6718 - tp: 11547.0000 - fp: 4303.0000 - tn: 197669.0000 - fn: 13989.0000
Epoch 4/30
3555/3555 [==============================] - 18s 5ms/step - loss: 0.1865 - accuracy: 0.9199 - precision: 0.7295 - recall: 0.4549 - auc: 0.9345 - prc: 0.6748 - tp: 11617.0000 - fp: 4307.0000 - tn: 197665.0000 - fn: 13919.0000
Epoch 5/30
3555/3555 [==============================] - 

####**Single Dataset**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded_1 = files.upload()

for fn in uploaded_1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_1[fn])))

Saving Client1.csv to Client1 (2).csv
User uploaded file "Client1.csv" with length 2266695 bytes


In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded_1['Client1.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

(22750, 24)

In [ ]:
Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape
Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))

original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

X_single = X
y_single = y

X: (22750, 13)
y: (22750,)
set()


#####Data Augmentation

In [ ]:
X = X_single
y = y_single

train(X, y, epochs = 20, batch_size = 128, latent_dim = 13)

# Save the generator
generator_0 = define_generator(latent_dim)

4/4 [==============================] - 0s 5ms/step
Epoch 0 Discriminator Loss: [0.7978478074073792, 0.34375] Generator Loss: 0.6932056546211243
4/4 [==============================] - 0s 5ms/step
Epoch 1 Discriminator Loss: [0.6936607360839844, 0.4453125] Generator Loss: 0.6932070255279541
4/4 [==============================] - 0s 5ms/step
Epoch 2 Discriminator Loss: [0.693413257598877, 0.52734375] Generator Loss: 0.6931804418563843
4/4 [==============================] - 0s 4ms/step
Epoch 3 Discriminator Loss: [0.6933383941650391, 0.53515625] Generator Loss: 0.6931934356689453
4/4 [==============================] - 0s 7ms/step
Epoch 4 Discriminator Loss: [0.6934952735900879, 0.46875] Generator Loss: 0.6931859254837036
4/4 [==============================] - 0s 4ms/step
Epoch 5 Discriminator Loss: [0.6934897303581238, 0.48046875] Generator Loss: 0.6931889057159424
4/4 [==============================] - 0s 6ms/step
Epoch 6 Discriminator Loss: [0.6936196088790894, 0.47265625] Generator Loss

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

In [ ]:
# Use the generator to create synthetic data
synthetic_data = generator_0.predict(np.random.normal(0, 1, (len(X_train), latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced = np.concatenate([X_train, synthetic_data])
y_balanced = np.concatenate([y_train, np.zeros(len(X_train))])

print("X_train", X_train.shape)
print("X_balanced", X_balanced.shape)

print("X_balanced:", X_balanced.shape)
print("y_balanced:", y_balanced.shape)

569/569 [==============================] - 2s 3ms/step
X_train (18200, 13)
X_balanced (36400, 13)
X_balanced: (36400, 13)
y_balanced: (36400,)


#####Centralized Training

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

model = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_shape=(13,)),
layers.Dense(64, activation='relu'),
layers.Dense(32, activation='relu'),
layers.Dense(1, activation='sigmoid')
])


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.fit(X_balanced, y_balanced, epochs=Epochs, batch_size=Batch_size)

Epoch 1/30
569/569 [==============================] - 5s 2ms/step - loss: 0.2285 - tp: 1737.0000 - fp: 952.0000 - tn: 31318.0000 - fn: 2393.0000 - accuracy: 0.9081 - precision: 0.6460 - recall: 0.4206 - auc: 0.8990 - prc: 0.5624
Epoch 2/30
569/569 [==============================] - 1s 3ms/step - loss: 0.1928 - tp: 1937.0000 - fp: 846.0000 - tn: 31424.0000 - fn: 2193.0000 - accuracy: 0.9165 - precision: 0.6960 - recall: 0.4690 - auc: 0.9311 - prc: 0.6576
Epoch 3/30
569/569 [==============================] - 2s 4ms/step - loss: 0.1912 - tp: 1945.0000 - fp: 808.0000 - tn: 31462.0000 - fn: 2185.0000 - accuracy: 0.9178 - precision: 0.7065 - recall: 0.4709 - auc: 0.9324 - prc: 0.6604
Epoch 4/30
569/569 [==============================] - 2s 4ms/step - loss: 0.1897 - tp: 1953.0000 - fp: 786.0000 - tn: 31484.0000 - fn: 2177.0000 - accuracy: 0.9186 - precision: 0.7130 - recall: 0.4729 - auc: 0.9335 - prc: 0.6672
Epoch 5/30
569/569 [==============================] - 2s 3ms/step - loss: 0.1884 - t

In [ ]:
Evaluation = model.evaluate(X_test, y_test)

143/143 [==============================] - 2s 2ms/step - loss: 0.3697 - tp: 488.0000 - fp: 190.0000 - tn: 3338.0000 - fn: 534.0000 - accuracy: 0.8409 - precision: 0.7198 - recall: 0.4775 - auc: 0.8538 - prc: 0.6842
